In [1]:
import datetime
import pandas_datareader.data as web
import pandas as pd
import nasdaqdatalink
import requests
import csv
from lxml import html
nasdaqdatalink.ApiConfig.api_key ='ABH2M7eCbJJXy7YS_Cj7'

In [2]:
# args year/month/day
startDate = datetime.datetime(2010,1,1)
endDate = datetime.datetime(2017,5,1)

index = web.DataReader('SPY', 'yahoo', startDate, endDate)
oil = nasdaqdatalink.get('FRED/DCOILWTICO', start_date = startDate, end_date = endDate)
naturalGas = nasdaqdatalink.get('FRED/DHHNGSP', start_date = startDate, end_date = endDate)
gold = nasdaqdatalink.get('LBMA/GOLD', start_date = startDate, end_date = endDate)

index = index['Adj Close'].pct_change()
oil = oil.pct_change()
gold = gold['USD (AM)'].pct_change()
naturalGas = naturalGas.pct_change()

In [3]:
naturalGas.rename(columns={'Value':'NaturalGas'},inplace=True)

In [4]:
df = pd.concat([index, oil, gold,naturalGas], axis = 1)
df.dropna(inplace=True)
df.rename(columns={'Value': 'Oil','USD (AM)':'Gold'}, inplace=True)
df.to_csv("Variables.csv")

In [5]:
# download page
page = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
# tree now contains the whole HTML file in a nice tree structure which we can go over two different ways: XPath and CSSSelect
tree = html.fromstring(page.content)
# print(page.content)

In [6]:
# xpath query
# inspect component then Copy Xpath
body = tree.xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table[1]/tbody')[0]
rows = body.findall('tr')
ticker = []
# We get rid of the first row because it is the header by indexing the array starting at 1
for x in rows[1:]:
    td_list = x.find("td")
    # text_content() returns a string of the text in the element
    ticker.append(td_list.text_content().strip())

In [7]:
with open('ticker.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(ticker)

In [8]:
body = tree.xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table[1]/tbody')[0]
rows = body.findall('tr')
rows = rows[1:]

In [9]:
df = pd.DataFrame()
for row in rows:
    placeholder = row.findall('td')
    content = [x.text_content().strip() for x in placeholder]
    df = pd.concat([df,pd.Series(content)],axis=1,ignore_index=True)

df = df.transpose()

df.columns = ["Ticker","Security","SEC Filings","GICS Sector","GICS Sub Industry","Location","Date Added","CIK","Founded"]    

df.to_csv("SP500.csv",index=False)